In [ ]:
# Ensure all necessary libraries are installed
!pip install pandas numpy matplotlib seaborn scikit-learn

# Import essential libraries for data manipulation and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
os.makedirs('data', exist_ok=True)


# Import machine learning modules
#from sklearn.model_selection import train_test_split, cross_val_score
#from sklearn.linear_model import LinearRegression
#from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
#from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
# Download the zip file from GitHub
!wget -O car_price_data.zip "https://github.com/SripathiVR/Car_Price_Prediction/blob/main/Car_Price%20Prediction.zip?raw=true"

# Unzip the downloaded file
!unzip car_price_data.zip -d ./car_price_data

--2024-11-28 12:47:15--  https://github.com/SripathiVR/Car_Price_Prediction/blob/main/Car_Price%20Prediction.zip?raw=true
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/SripathiVR/CarDekho-Insights-Predicting-Used-Car-Prices-with-Streamlit-and-ML/blob/main/Car_Price%20Prediction.zip?raw=true [following]
--2024-11-28 12:47:15--  https://github.com/SripathiVR/CarDekho-Insights-Predicting-Used-Car-Prices-with-Streamlit-and-ML/blob/main/Car_Price%20Prediction.zip?raw=true
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/SripathiVR/CarDekho-Insights-Predicting-Used-Car-Prices-with-Streamlit-and-ML/raw/refs/heads/main/Car_Price%20Prediction.zip [following]
--2024-11-28 12:47:15--  https://github.com/SripathiVR/CarDekho-Insights-Predicting-Used-Car-Prices-

In [ ]:
import zipfile
import requests
import io

# Step 1: Download the zip file from GitHub
url = 'https://github.com/SripathiVR/Car_Price_Prediction/raw/main/Car_Price%20Prediction.zip'
response = requests.get(url)
zip_file = zipfile.ZipFile(io.BytesIO(response.content))

# Step 2: Check the contents of the zip file
print("Files in zip:", zip_file.namelist())  # List all files in the zip


# Step 3: Load the CSV file (replace with actual CSV name if different)

try:
    # Adjust the file path to the specific CSV inside the zip structure
    data = pd.read_csv(zip_file.open('Car_Price Prediction/car_dekho_cleaned_dataset_Raw.csv'))
    print(data.head())
except KeyError:
    print("Specified file was not found in the zip archive.")
except pd.errors.EmptyDataError:
    print("The file appears to be empty.")



Files in zip: ['Car_Price Prediction/', 'Car_Price Prediction/Car_Dataset/', 'Car_Price Prediction/Car_Dataset/bangalore_cars.xlsx', 'Car_Price Prediction/Car_Dataset/chennai_cars.xlsx', 'Car_Price Prediction/Car_Dataset/delhi_cars.xlsx', 'Car_Price Prediction/Car_Dataset/hyderabad_cars.xlsx', 'Car_Price Prediction/Car_Dataset/jaipur_cars.xlsx', 'Car_Price Prediction/Car_Dataset/kolkata_cars.xlsx', 'Car_Price Prediction/Car_Dekho-Used_Car_Price_Prediction.pdf', 'Car_Price Prediction/Car_Dekho.ipynb', 'Car_Price Prediction/car_dekho_app.py', 'Car_Price Prediction/car_dekho_cleaned_dataset.csv', 'Car_Price Prediction/car_dekho_cleaned_dataset_Raw.csv', 'Car_Price Prediction/car_dekho_Structured.csv', 'Car_Price Prediction/Feature Description.docx']
   it      ft         bt        km transmission  ownerNo      oem  \
0   0  Petrol  Hatchback  120000.0       Manual        3   Maruti   
1   0  Petrol        SUV   32706.0       Manual        2     Ford   
2   0  Petrol  Hatchback   11949.0  

<ipython-input-3-36e756fa3ab9>:18: DtypeWarning: Columns (155) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(zip_file.open('Car_Price Prediction/car_dekho_cleaned_dataset_Raw.csv'))


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Step 1: Remove Columns with Excessive Missing Values
missing_threshold = len(data) * 0.3
cleaned_data = data.dropna(thresh=missing_threshold, axis=1)

# Step 2: Fill in Missing Data
for feature in cleaned_data.columns:
    # Determine if the column is categorical
    if cleaned_data[feature].dtype == 'object':
        # Replace missing values with the mode (most common value)
        most_common_value = cleaned_data[feature].mode()[0]
        cleaned_data[feature].fillna(most_common_value, inplace=True)
    else:
        # Replace missing values with the mean of the column
        mean_value = cleaned_data[feature].mean()
        cleaned_data[feature].fillna(mean_value, inplace=True)

# Step 3: Eliminate Non-Essential Columns
unnecessary_columns = ['car_links', 'City']  # Update this list as required
cleaned_data = cleaned_data.drop(columns=unnecessary_columns, errors='ignore')

# Step 4: Transform Categorical Features
# Apply Label Encoding to columns with high cardinality
high_cardinality_features = [feature for feature in cleaned_data.select_dtypes(include=['object']).columns if cleaned_data[feature].nunique() > 100]
for feature in high_cardinality_features:
    label_encoder = LabelEncoder()
    cleaned_data[feature] = label_encoder.fit_transform(cleaned_data[feature].astype(str))

# One-Hot Encode low-cardinality categorical features
low_cardinality_features = [feature for feature in cleaned_data.select_dtypes(include=['object']).columns if feature not in high_cardinality_features]
cleaned_data = pd.get_dummies(cleaned_data, drop_first=True, columns=low_cardinality_features)

# Display the structure of the cleaned dataset
print("\nSummary of Cleaned Data:")
cleaned_data.info()

<ipython-input-4-b46304e204ee>:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  cleaned_data[feature].fillna(mean_value, inplace=True)
<ipython-input-4-b46304e204ee>:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', tr


Summary of Cleaned Data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7511 entries, 0 to 7510
Columns: 2023 entries, it to data_2_list_11_key_No Door Numbers
dtypes: bool(1982), float64(6), int64(35)
memory usage: 16.5 MB


In [ ]:
from scipy.stats import zscore

# Step 5: Check for remaining missing values
print("Remaining Missing Values:\n", cleaned_data.isnull().sum()[cleaned_data.isnull().sum() > 0])

# Summary of data types
print("\nData Types Summary:\n", cleaned_data.dtypes.value_counts())

# Outliers analysis for numeric columns
outliers = {}
for col in cleaned_data.select_dtypes(include=['float64', 'int64']).columns:
    outliers[col] = (zscore(cleaned_data[col].fillna(0)) > 3).sum()  # Count of outliers
print("\nOutliers Count in Numerical Columns:\n", outliers)

# Detect columns with single unique values (likely redundant)
try:
    redundant_columns = [col for col in cleaned_data.columns if cleaned_data[col].nunique(dropna=False) == 1]
    print("\nRedundant Columns (Single Unique Value):\n", redundant_columns)

    # Optionally, drop redundant columns
    cleaned_data = cleaned_data.drop(columns=redundant_columns)
    print("\nRedundant columns removed. Final Data Shape:", cleaned_data.shape)

except Exception as e:
    print("An error occurred:", e)

Remaining Missing Values:
 Series([], dtype: int64)

Data Types Summary:
 bool       1982
int64        35
float64       6
Name: count, dtype: int64

Outliers Count in Numerical Columns:
 {'it': 0, 'km': 14, 'ownerNo': 97, 'model': 0, 'modelYear': 0, 'centralVariantId': 0, 'variantName': 0, 'price': 55, 'top_0_value': 0, 'top_4_value': 0, 'top_5_value': 0, 'top_6_value': 0, 'top_7_value': 0, 'top_9_value': 0, 'top_1_value.2': 0, 'top_2_value.2': 0, 'top_3_value.2': 0, 'data_0_list_0_value.1': 0, 'data_0_list_1_value.1': 0, 'data_0_list_2_value.1': 0, 'data_0_list_3_value.1': 0, 'data_0_list_4_value.1': 0, 'data_0_list_8_value.1': 0, 'data_0_list_9_value.1': 0, 'data_1_list_0_value.1': 25, 'data_1_list_1_value.1': 52, 'data_1_list_2_value.1': 0, 'data_1_list_3_value.1': 29, 'data_1_list_4_value.1': 0, 'data_1_list_5_value.1': 0, 'data_1_list_6_value.1': 0, 'data_2_list_4_value.1': 0, 'data_2_list_6_value.1': 0, 'data_2_list_7_value.1': 0, 'data_2_list_8_value.1': 0, 'data_2_list_9_value.

In [ ]:
save_path = '/content/data/cleaned_data.csv'
cleaned_data.to_csv(save_path, index=False)
# Make sure to create the data directory first

os.makedirs('data', exist_ok=True)

# Save the cleaned data
cleaned_data.to_csv('data/cleaned_data.csv', index=False)

# Verify if the file is saved correctly
if os.path.exists('data/cleaned_data.csv'):
       print(f"Data saved successfully to {save_path}")
else:
    print("Error: Data file was not saved.")

Data saved successfully to /content/data/cleaned_data.csv


In [ ]:
print("Files in '/content/data' directory:", os.listdir('/content/data'))

Files in '/content/data' directory: ['cleaned_data.csv']


In [ ]:
cleaned_data.to_csv('/content/data/cleaned_data.csv', index=False)